In [49]:
%%script false --no-raise-error
#Copy this code and run it in the terminal everytime you create a new directory or file
#Gives permission for everyone to work with the files created
#Run at the end of a jupyter script, the output is too messy
!cd /cephyr/NOBACKUP/groups/bbt045_2025/groups/group_Bjork 
!chmod -R 777 . 

# Download cDNA library 
From ensembl: https://ftp.ensembl.org/pub/release-113/fasta/homo_sapiens/cdna/ and unzip.

In [17]:
%%bash
# -nc makes sure that if the file is already there, it won't be overwritten.
# -nc can be changed to -N, if it is of interest to import newer versions
# -P will save it to the directory specified, in this case data/cDNA_library
wget -nc ftp://ftp.ensembl.org/pub/release-113/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz -P data/
gzip -d data/Homo_sapiens.GRCh38.cdna.all.fa.gz
rm data/Homo_sapiens.GRCh38.cdna.all.fa.gz

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory
File 'data/Homo_sapiens.GRCh38.cdna.all.fa.gz' already there; not retrieving.
gzip: data/Homo_sapiens.GRCh38.cdna.all.fa already exists;	not overwritten


In [ ]:
!ls data

# Build Bowtie index
This takes around 30 minutes to run on one kernel, so don't run it if you don't have to.

In [20]:
%%bash
mkdir data/Bowtie_index
bowtie2-build data/Homo_sapiens.GRCh38.cdna.all.fa data/Bowtie_index/hg38_cdna_index

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


Settings:
  Output files: "data/Bowtie_index/hg38_cdna_index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  data/Homo_sapiens.GRCh38.cdna.all.fa


Building a SMALL index


Reading reference sizes
  Time reading reference sizes: 00:00:03
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:02
bmax according to bmaxDivN setting: 98755563
Using parameters --bmax 74066673 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 74066673 --dcv 1024
Constructing suffix-array element generator
Building DifferenceCoverSample
  Building sPrime
  Building sPrimeOrder
  V-Sorting samples
  V-Sorting samples time: 00:00:32
  Allocating rank array
  Ranking v-sort output
  Ranking v-sort output time: 00:00:04
  Invoking Larsson-Sadakane on ranks
  Invoking Larsson-Sadakane on ranks time: 00:00:03
  Sanity-checking and returning
Building samples
Reserving space for 12 sample suffixes
Generating random suffixes
QSorting 12 sample offsets, eliminating duplicates
QSorting sample offsets, eliminating duplicates time: 00:00:00
Multikey

Renaming data/Bowtie_index/hg38_cdna_index.3.bt2.tmp to data/Bowtie_index/hg38_cdna_index.3.bt2
Renaming data/Bowtie_index/hg38_cdna_index.4.bt2.tmp to data/Bowtie_index/hg38_cdna_index.4.bt2
Renaming data/Bowtie_index/hg38_cdna_index.1.bt2.tmp to data/Bowtie_index/hg38_cdna_index.1.bt2
Renaming data/Bowtie_index/hg38_cdna_index.2.bt2.tmp to data/Bowtie_index/hg38_cdna_index.2.bt2
Renaming data/Bowtie_index/hg38_cdna_index.rev.1.bt2.tmp to data/Bowtie_index/hg38_cdna_index.rev.1.bt2
Renaming data/Bowtie_index/hg38_cdna_index.rev.2.bt2.tmp to data/Bowtie_index/hg38_cdna_index.rev.2.bt2


In [21]:
!ls data/subset_trimmed_RNA-seq/

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory
ERR950158_trimmed_1.fastq.gz  ERR950178_trimmed_2.fastq.gz
ERR950158_trimmed_2.fastq.gz  ERR950179_trimmed_1.fastq.gz
ERR950159_trimmed_1.fastq.gz  ERR950179_trimmed_2.fastq.gz
ERR950159_trimmed_2.fastq.gz  ERR950180_trimmed_1.fastq.gz
ERR950160_trimmed_1.fastq.gz  ERR950180_trimmed_2.fastq.gz
ERR950160_trimmed_2.fastq.gz  ERR950181_trimmed_1.fastq.gz
ERR950161_trimmed_1.fastq.gz  ERR950181_trimmed_2.fastq.gz
ERR950161_trimmed_2.fastq.gz  ERR950182_trimmed_1.fastq.gz
ERR950162_trimmed_1.fastq.gz  ERR950182_trimmed_2.fastq.gz
ERR950162_trimmed_2.fastq.gz  ERR950186_trimmed_1.fastq.gz
ERR950163_trimmed_1.fastq.gz  ERR950186_trimmed_2.fastq.gz
ERR950163_trimmed_2.fastq.gz  ERR950187_trimmed_1.fastq.gz
ERR950177_trimmed_1.fastq.gz  ERR950187_trimmed_2.fastq.gz
ERR950177_trimmed_2.fastq.gz  fastqc_subset_trimmed
ERR950178_trimmed_1.fastq.gz


In [30]:
%%bash

mkdir -p alignment_Bowtie2_subset

# Loop over all FASTQ files ending in _1.fastq.gz in the subset_trimmed_RNA-seq folder
for R1 in data/subset_trimmed_RNA-seq/*_1.fastq.gz; do
  
  # Extract the base name 
  BN=$(basename "$R1")
  
  # Remove the _1.fastq.gz suffix to get the sample prefix 
  SAMPLE=${BN%%_1.fastq.gz}
  
  # Construct the path to the mate 2 file 
  R2="data/subset_trimmed_RNA-seq/${SAMPLE}_2.fastq.gz"
  
  echo "Aligning $SAMPLE with Bowtie2..."
  
  # Run Bowtie2 in local alignment mode for the paired-end reads
  bowtie2 --local 
    -x data/hg38_cdna_index 
    -1 "$R1" 
    -2 "$R2" 
    -S alignment_Bowtie2_subset/${SAMPLE}.sam 
    --threads 4
  
  # Convert the SAM to a sorted BAM and create an index using samtools
  samtools view -bS alignment_Bowtie2_subset/${SAMPLE}.sam | samtools sort -o alignment_Bowtie2_subset/${SAMPLE}.bam
  samtools index alignment_Bowtie2_subset/${SAMPLE}.bam
  
  # Remove the SAM file to save disk space
  rm alignment_Bowtie2_subset/${SAMPLE}.sam
  
  echo "Done aligning $SAMPLE"
  echo "-----------------------------------------"
done


/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


Aligning ERR950158_trimmed with Bowtie2...


No index, query, or output file specified!
Bowtie 2 version 2.5.4 by Ben Langmead (langmea@cs.jhu.edu, www.cs.jhu.edu/~langmea)
Usage: 
  bowtie2 [options]* -x <bt2-idx> {-1 <m1> -2 <m2> | -U <r> | --interleaved <i> | -b <bam>} [-S <sam>]

  <bt2-idx>  Index filename prefix (minus trailing .X.bt2).
             NOTE: Bowtie 1 and Bowtie 2 indexes are not compatible.
  <m1>       Files with #1 mates, paired with files in <m2>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <m2>       Files with #2 mates, paired with files in <m1>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <r>        Files with unpaired reads.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <i>        Files with interleaved paired-end FASTQ/FASTA reads
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <bam>      Files are unaligned BAM sorted by read name.
  <sam>      File for SAM output (

Done aligning ERR950158_trimmed
-----------------------------------------
Aligning ERR950159_trimmed with Bowtie2...


No index, query, or output file specified!
Bowtie 2 version 2.5.4 by Ben Langmead (langmea@cs.jhu.edu, www.cs.jhu.edu/~langmea)
Usage: 
  bowtie2 [options]* -x <bt2-idx> {-1 <m1> -2 <m2> | -U <r> | --interleaved <i> | -b <bam>} [-S <sam>]

  <bt2-idx>  Index filename prefix (minus trailing .X.bt2).
             NOTE: Bowtie 1 and Bowtie 2 indexes are not compatible.
  <m1>       Files with #1 mates, paired with files in <m2>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <m2>       Files with #2 mates, paired with files in <m1>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <r>        Files with unpaired reads.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <i>        Files with interleaved paired-end FASTQ/FASTA reads
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <bam>      Files are unaligned BAM sorted by read name.
  <sam>      File for SAM output (

...

Done aligning ERR950182_trimmed
-----------------------------------------
Aligning ERR950186_trimmed with Bowtie2...


No index, query, or output file specified!
Bowtie 2 version 2.5.4 by Ben Langmead (langmea@cs.jhu.edu, www.cs.jhu.edu/~langmea)
Usage: 
  bowtie2 [options]* -x <bt2-idx> {-1 <m1> -2 <m2> | -U <r> | --interleaved <i> | -b <bam>} [-S <sam>]

  <bt2-idx>  Index filename prefix (minus trailing .X.bt2).
             NOTE: Bowtie 1 and Bowtie 2 indexes are not compatible.
  <m1>       Files with #1 mates, paired with files in <m2>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <m2>       Files with #2 mates, paired with files in <m1>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <r>        Files with unpaired reads.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <i>        Files with interleaved paired-end FASTQ/FASTA reads
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <bam>      Files are unaligned BAM sorted by read name.
  <sam>      File for SAM output (

Done aligning ERR950186_trimmed
-----------------------------------------
Aligning ERR950187_trimmed with Bowtie2...


No index, query, or output file specified!
Bowtie 2 version 2.5.4 by Ben Langmead (langmea@cs.jhu.edu, www.cs.jhu.edu/~langmea)
Usage: 
  bowtie2 [options]* -x <bt2-idx> {-1 <m1> -2 <m2> | -U <r> | --interleaved <i> | -b <bam>} [-S <sam>]

  <bt2-idx>  Index filename prefix (minus trailing .X.bt2).
             NOTE: Bowtie 1 and Bowtie 2 indexes are not compatible.
  <m1>       Files with #1 mates, paired with files in <m2>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <m2>       Files with #2 mates, paired with files in <m1>.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <r>        Files with unpaired reads.
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <i>        Files with interleaved paired-end FASTQ/FASTA reads
             Could be gzip'ed (extension: .gz) or bzip2'ed (extension: .bz2).
  <bam>      Files are unaligned BAM sorted by read name.
  <sam>      File for SAM output (

Done aligning ERR950187_trimmed
-----------------------------------------


In [27]:
!ls -l alignment_Bowtie2_subset/

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory
total 1464931
-rw-r--r-- 1 nobody nogroup  83164222 Feb 25 20:59 ERR950158_trimmed.bam
-rw-r--r-- 1 nobody nogroup   6049096 Feb 26 10:08 ERR950158_trimmed.bam.bai
-rw-r--r-- 1 nobody nogroup  82982403 Feb 25 21:02 ERR950159_trimmed.bam
-rw-r--r-- 1 nobody nogroup   5178176 Feb 26 10:08 ERR950159_trimmed.bam.bai
-rw-r--r-- 1 nobody nogroup  83563426 Feb 25 21:05 ERR950160_trimmed.bam
-rw-r--r-- 1 nobody nogroup   5964464 Feb 26 10:08 ERR950160_trimmed.bam.bai
-rw-r--r-- 1 nobody nogroup  80792165 Feb 25 21:08 ERR950161_trimmed.bam
-rw-r--r-- 1 nobody nogroup   5849536 Feb 26 10:08 ERR950161_trimmed.bam.bai
-rw-r--r-- 1 nobody nogroup  69514274 Feb 25 21:11 ERR950162_trimmed.bam
-rw-r--r-- 1 nobody nogroup   4070304 Feb 26 10:08 ERR950162_trimmed.bam.bai
-rw-r--r-- 1 nobody nogroup  84502479 Feb 25 21:14 ERR950163_trimmed.bam
-rw-r--r-- 1 nobody nogroup   5878184 

In [29]:
%%bash
samtools flagstat alignment_Bowtie2_subset/ERR950187_trimmed.bam

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


998450 + 0 in total (QC-passed reads + QC-failed reads)
998450 + 0 primary
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
450566 + 0 mapped (45.13% : N/A)
450566 + 0 primary mapped (45.13% : N/A)
998450 + 0 paired in sequencing
499225 + 0 read1
499225 + 0 read2
422530 + 0 properly paired (42.32% : N/A)
438398 + 0 with itself and mate mapped
12168 + 0 singletons (1.22% : N/A)
11068 + 0 with mate mapped to a different chr
4064 + 0 with mate mapped to a different chr (mapQ>=5)


/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


ERR950158_trimmed.bam
ERR950158_trimmed.bam.bai
ERR950159_trimmed.bam
ERR950159_trimmed.bam.bai
ERR950160_trimmed.bam
ERR950160_trimmed.bam.bai
ERR950161_trimmed.bam
ERR950161_trimmed.bam.bai
ERR950162_trimmed.bam
ERR950162_trimmed.bam.bai
ERR950163_trimmed.bam
ERR950163_trimmed.bam.bai
ERR950177_trimmed.bam
ERR950177_trimmed.bam.bai
ERR950178_trimmed.bam
ERR950178_trimmed.bam.bai
ERR950179_trimmed.bam
ERR950179_trimmed.bam.bai
ERR950180_trimmed.bam
ERR950180_trimmed.bam.bai
ERR950181_trimmed.bam
ERR950181_trimmed.bam.bai
ERR950182_trimmed.bam
ERR950182_trimmed.bam.bai
ERR950186_trimmed.bam
ERR950186_trimmed.bam.bai
ERR950187.log
ERR950187_trimmed.bam
ERR950187_trimmed.bam.bai
ERR950187_trimmed_1.sam
